In [1]:
import os
import sys

path = os.getcwd()
par_path = os.path.abspath(os.pardir)
sys.path.append(par_path)

from utils.data import ExtractROI, AtlasDataset

import matplotlib.pyplot as plt
import numpy as np
import torch

import math
import random as rand
from tqdm import tqdm

from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

## **Extract ROIs**

You will need to manually delete the save directories in order to regenerate to ROIs (as a precaution against accidentally running and overwriting). 

In [3]:
ROI_size = (64, 64)
sampling_rate = 0.5
str_sampling_rate = str(sampling_rate).replace('.', '_')

extract_dir = '/nafs/dtward/allen/npz_files/'
save_dir = '/nafs/dtward/allen/rois/'
sub_save_dir = f"{ROI_size[0]}x{ROI_size[1]}_sample_rate_{str_sampling_rate}_center/"
abs_save_dir = os.path.join(save_dir, sub_save_dir)

if not os.path.exists(abs_save_dir):
    os.makedirs(abs_save_dir)
    extractor_center = ExtractROI(extract_dir=extract_dir, save_dir=abs_save_dir)
    extractor_center.extract_ROI(ROI_size=ROI_size, sampling_rate=sampling_rate, label_mode='center')

ROI_size = (64, 64)
sampling_rate = 0.5
str_sampling_rate = str(sampling_rate).replace('.', '_')

extract_dir = '/nafs/dtward/allen/npz_files/'
save_dir = '/nafs/dtward/allen/rois/'
sub_save_dir = f"{ROI_size[0]}x{ROI_size[1]}_sample_rate_{str_sampling_rate}_all/"
abs_save_dir = os.path.join(save_dir, sub_save_dir)

if not os.path.exists(abs_save_dir):
    os.makedirs(abs_save_dir)
    extractor_all = ExtractROI(extract_dir=extract_dir, save_dir=abs_save_dir)
    extractor_all.extract_ROI(ROI_size=ROI_size, sampling_rate=sampling_rate, label_mode='all')

ROI_size = (64, 64)
sampling_rate = 0.5
str_sampling_rate = str(sampling_rate).replace('.', '_')

extract_dir = '/nafs/dtward/allen/npz_files/'
save_dir = '/nafs/dtward/allen/rois/'
sub_save_dir = f"{ROI_size[0]}x{ROI_size[1]}_sample_rate_{str_sampling_rate}_mode/"
abs_save_dir = os.path.join(save_dir, sub_save_dir)

if not os.path.exists(abs_save_dir):
    os.makedirs(abs_save_dir)
    extractor_all = ExtractROI(extract_dir=extract_dir, save_dir=abs_save_dir)
    extractor_all.extract_ROI(ROI_size=ROI_size, sampling_rate=sampling_rate, label_mode='mode')

100%|██████████| 213/213 [07:19<00:00,  2.06s/it]

All ROIs extracted from /nafs/dtward/allen/npz_files/ and saved to /nafs/dtward/allen/rois/64x64_sample_rate_0_5_mode/ under train and test subfolders


## **Dataloader**

In [19]:
train_dir = '/nafs/dtward/allen/rois/64x64_sample_rate_0_5_mode/train/'
test_dir = '/nafs/dtward/allen/rois/64x64_sample_rate_0_5_mode/test/'

train_dataset = AtlasDataset(data_dir=train_dir)
test_dataset = AtlasDataset(data_dir=test_dir)

bs = 32
train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=bs, shuffle=True)